In [9]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [13]:
print(os.path.exists('/Users/weimingchin/Desktop/term 7/Spatial Design/spatialworld/image-segmentation/example.jpg'))


True


In [14]:
image = cv2.imread('/Users/weimingchin/Desktop/term 7/Spatial Design/spatialworld/image-segmentation/example.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [15]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
predictor.set_image(image)

In [ ]:
import numpy as np

# Function to collect input points from the user
def get_input_point():
    points = []

    while True:
        # Get user input
        user_input = input("Enter a point as x,y (or type 'done' to finish): ")

        if user_input.lower() == 'done':
            break

        # Split and convert the input to integers
        try:
            x, y = map(int, user_input.split(','))
            points.append([x, y])
        except ValueError:
            print("Invalid input. Please enter coordinates in the format x,y.")

    return np.array(points)

# Function to collect input labels from the user
def get_input_label(num_points):
    labels = []

    for i in range(num_points):
        while True:
            try:
                label = int(input(f"Enter label for point {i+1} (1 for positive, 0 for negative): "))
                if label in [0, 1]:  # Label should be 0 or 1
                    labels.append(label)
                    break
                else:
                    print("Label must be 0 or 1.")
            except ValueError:
                print("Invalid input. Please enter 0 or 1.")

    return np.array(labels)

# Get input points from user
input_point = get_input_point()

# Get input labels for each point
input_label = get_input_label(len(input_point))

# Print the result
print("Final input_point array:", input_point)
print("Final input_label array:", input_label)

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)

mask_input = logits[np.argmax(scores), :, :]  # Choose the model's best mask

masks, _, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)

masks.shape

plt.figure(figsize=(10,10))
plt.imshow(image)
show_mask(masks, plt.gca())
show_points(input_point, input_label, plt.gca())
plt.axis('off')
plt.show()